## Summary

---

## Imports

In [ ]:
import os
from pathlib import Path

import crc32c
import pyarrow as pa
import pyarrow.parquet as pq
from tqdm.notebook import tqdm

## Parameters

In [ ]:
NOTEBOOK_NAME = "01_load_data"

In [ ]:
NOTEBOOK_DIR = Path(NOTEBOOK_NAME).resolve()
NOTEBOOK_DIR.mkdir(exist_ok=True)

NOTEBOOK_DIR

In [ ]:
if "DATAPKG_OUTPUT_DIR" in os.environ:
    DATAPKG_OUTPUT_DIR = Path(os.getenv("DATAPKG_OUTPUT_DIR")).resolve()
else:
    DATAPKG_OUTPUT_DIR = NOTEBOOK_DIR
DATAPKG_OUTPUT_DIR.mkdir(exist_ok=True)

DATAPKG_OUTPUT_DIR

In [ ]:
if "DATAPKG_OUTPUT_DIR" in os.environ:
    OUTPUT_DIR = Path(os.getenv("DATAPKG_OUTPUT_DIR")).joinpath("elaspic-v2").resolve()
else:
    OUTPUT_DIR = NOTEBOOK_DIR.parent
OUTPUT_DIR.mkdir(exist_ok=True)

OUTPUT_DIR

## Datasets

In [ ]:
resources = {
    # === Core ===
    "elaspic-training-set-core": DATAPKG_OUTPUT_DIR.joinpath(
        "elaspic-training-set", "02_export_data_core", "elaspic-training-set-core.parquet"
    ),
    "protherm-dagger-core": DATAPKG_OUTPUT_DIR.joinpath(
        "protein-folding-energy", "protherm_dagger", "mutation-by-sequence.parquet"
    ),
    "rocklin-2017-core": DATAPKG_OUTPUT_DIR.joinpath(
        "protein-folding-energy", "rocklin_2017", "mutation-ssm2.parquet"
    ),
    # "dunham-2020-core": DATAPKG_OUTPUT_DIR.joinpath(
    #     "protein-folding-energy", "dunham_2020", "dunham-core.parquet"
    # ),
    # === Interface ===
    "elaspic-training-set-interface": DATAPKG_OUTPUT_DIR.joinpath(
        "elaspic-training-set", "02_export_data_interface", "elaspic-training-set-interface.parquet"
    ),
    "skempi-v2-interface": DATAPKG_OUTPUT_DIR.joinpath(
        "protein-folding-energy", "skempi_v2", "skempi-v2.parquet"
    ),
    # "intact-mutations-interface": DATAPKG_OUTPUT_DIR.joinpath(
    #     "protein-folding-energy", "intact_mutations", "intact-mutations.parquet"
    # ),
    # "dunham-2020-interface": DATAPKG_OUTPUT_DIR.joinpath(
    #     "protein-folding-energy", "dunham_2020", "dunham-interface.parquet"
    # ),
}

In [ ]:
for name, path in resources.items():
    assert Path(path).is_file(), path

## Load data

In [ ]:
columns = [
    "unique_id",
    "dataset",
    "name",
    "protein_sequence",
    "ligand_sequence",
    "mutation",
    "effect",
    "effect_type",
    "protein_structure",
]

extra_columns = [
    "provean_score",
    "foldx_score",
    "elaspic_score",
]

In [ ]:
def get_unique_id(dataset, effect_type, protein_sequence, ligand_sequence):
    if ligand_sequence is not None:
        key = f"{dataset}|{effect_type}|{protein_sequence}|{ligand_sequence}"
    else:
        key = f"{dataset}|{effect_type}|{protein_sequence}"
    return crc32c.crc32c(key.encode("utf-8"))

In [ ]:
output_dir = OUTPUT_DIR.joinpath(NOTEBOOK_NAME).resolve()
output_dir.mkdir(exist_ok=True)

output_dir

In [ ]:
_seen = {
    "core": set(),
    "interface": set(),
}

for dataset_name, dataset_file in resources.items():
    coi = dataset_name.rsplit("-", 1)[-1]
    assert coi in ["core", "interface"]

    df = (
        pq.read_table(dataset_file)
        .to_pandas(integer_object_nulls=True)
        .rename(columns={"mutations": "mutation"})
    )
    print(f"Read {len(df)} rows.")

    # Remove unneeded data
    mask = df["mutation"].apply(len) >= 2
    print(f"Removing {(~mask).sum()} rows with fewer than two mutations.")
    df = df[mask]

    mask = df["effect"].apply(lambda x: len(set(x))) >= 2
    print(f"Removing {(~mask).sum()} rows with fewer than two unique effects.")
    df = df[mask]

    if "dataset" not in df:
        df["dataset"] = dataset_name
    if "ligand_sequence" not in df:
        df["ligand_sequence"] = None

    # Add a unique id
    df["unique_id"] = [
        get_unique_id(dataset, effect_type, protein_sequence, ligand_sequence)
        for dataset, effect_type, protein_sequence, ligand_sequence in df[
            ["dataset", "effect_type", "protein_sequence", "ligand_sequence"]
        ].values
    ]
    unique_ids = set(df["unique_id"].values)
    assert len(unique_ids) == len(df)
    assert not set(unique_ids) & _seen[coi]
    _seen[coi].update(unique_ids)

    columns_all = columns + [c for c in extra_columns if c in df]
    df_out = df[columns_all]

    # Write files
    row_per_batch = 100
    writer = None
    for start in tqdm(range(0, len(df_out), row_per_batch), desc=dataset_name):
        stop = start + row_per_batch
        df_out_slice = df_out.iloc[start:stop]
        table = pa.Table.from_pandas(df_out_slice, preserve_index=False)

        if writer is None:
            output_file = output_dir.joinpath(f"{dataset_name}.parquet")
            writer = pq.ParquetWriter(output_file, table.schema)
            print(output_file)

        writer.write_table(table)
    writer.close()

In [ ]:
raise Exception

In [ ]:
_seen = {
    "core": set(),
    "interface": set(),
}

for dataset_name, dataset_file in resources.items():
    coi = dataset_name.rsplit("-", 1)[-1]
    assert coi in ["core", "interface"]

    pfile = pq.ParquetFile(dataset_file)
    writer = None
    columns_all = None
    for row_group_idx in tqdm(range(pfile.num_row_groups), desc=dataset_name):
        df = (
            pfile.read_row_group(row_group_idx)
            .to_pandas(integer_object_nulls=True)
            .rename(columns={"mutations": "mutation"})
        )

        # Remove unneeded data
        mask = df["mutation"].apply(len) >= 2
        print(f"Removing {(~mask).sum()} rows with fewer than two mutations.")
        df = df[mask]

        mask = df["effect"].apply(lambda x: len(set(x))) >= 2
        print(f"Removing {(~mask).sum()} rows with fewer than two unique effects.")
        df = df[mask]

        if "dataset" not in df:
            df["dataset"] = dataset_name
        if "ligand_sequence" not in df:
            df["ligand_sequence"] = None

        # Add a unique id
        df["unique_id"] = [
            get_unique_id(dataset, effect_type, protein_sequence, ligand_sequence)
            for dataset, effect_type, protein_sequence, ligand_sequence in df[
                ["dataset", "effect_type", "protein_sequence", "ligand_sequence"]
            ].values
        ]
        unique_ids = set(df["unique_id"].values)
        assert len(unique_ids) == len(df)
        assert not set(unique_ids) & _seen[coi]
        _seen[coi].update(unique_ids)

        # Add extra columns
        if columns_all is None:
            columns_all = columns + [c for c in extra_columns if c in df]

        df_out = df[columns_all]

        # Write files
        row_per_batch = 100
        for start in range(0, len(df_out), row_per_batch):
            stop = start + row_per_batch
            df_out_slice = df_out.iloc[start:stop]
            table = pa.Table.from_pandas(df_out_slice, preserve_index=False)

            if writer is None:
                output_file = output_dir.joinpath(f"{dataset_name}.parquet")
                writer = pq.ParquetWriter(output_file, table.schema)
                print(output_file)

            writer.write_table(table)
    writer.close()

In [ ]:
raise Exception

In [ ]:
pfile = pq.ParquetFile("/home/kimlab1/database_data/datapkg_output_dir/elaspic-v2/01_load_data_core/mutation-ssm2.parquet")

In [ ]:
_seen.update()

In [ ]:
df.head()

In [ ]:
df2.head()

In [ ]:
pfile.num_row_groups

In [ ]:
table = pfile.read_row_group

In [ ]:
pa.Table.to_pandas()

In [ ]:
30_000 / 300